In [45]:
import cv2
import numpy as np
import os
from skimage.feature import hog
from sklearn.svm import SVC
from sklearn.metrics import precision_score
import sys
del sys.modules["Funciones"]
import Funciones

In [103]:
# 1. Preprocesamiento
from Funciones import filtroMediana
def preprocesar_imagen(ruta_imagen):
    imagen = cv2.imread(ruta_imagen, cv2.IMREAD_GRAYSCALE)
    if imagen is None:  # Manejar imágenes no cargadas
        print(f"Error al cargar la imagen: {ruta_imagen}")
        return None
    imagen = cv2.resize(imagen, (128, 128))  # Redimensionar a 128x128
    #imagen = cv2.equalizeHist(imagen)        # Ecualización del histograma
    #imagen = Funciones.filtroMediana(imagen)
    #imagen = Funciones.gradiente(imagen)
    #imagen = imagen/np.max(imagen)
    #imagen_suavizada = cv2.GaussianBlur(imagen, (5, 5), 0)
    return imagen

In [104]:
# 2. Extracción de características HOG
def extraccion_hog(imagen):
    fd = hog(
        imagen,
        orientations=8,
        pixels_per_cell=(16, 16),
        cells_per_block=(1, 1),
        visualize=False  # Solo devuelve el descriptor
    )
    return fd

In [105]:
# 3. Creación del dataset
def crear_dataset(directorio_imagenes, extractor_caracteristicas):
    """
    Crea un dataset de características y etiquetas a partir de un directorio.
    """
    X, y = [], []  # X para características, y para etiquetas
    for etiqueta, nombre_clase in enumerate(['cat', 'dog']):
        directorio_clase = os.path.join(directorio_imagenes, nombre_clase)
        for nombre_archivo in os.listdir(directorio_clase):
            ruta_imagen = os.path.join(directorio_clase, nombre_archivo)
            imagen = preprocesar_imagen(ruta_imagen)
            if imagen is None:  # Omitir imágenes no válidas
                continue
            caracteristicas = extractor_caracteristicas(imagen)
            X.append(caracteristicas)
            y.append(etiqueta)
    return np.array(X), np.array(y)

In [118]:
def crear_dataset(directorio_imagenes, extractor_caracteristicas):
    """
    Crea un dataset de características y etiquetas a partir de un directorio.
    """
    X, y = [], []  # X para características, y para etiquetas
    for etiqueta, nombre_clase in enumerate(['cat', 'dog']):
        directorio_clase = os.path.join(directorio_imagenes, nombre_clase)
        for nombre_archivo in os.listdir(directorio_clase):
            ruta_imagen = os.path.join(directorio_clase, nombre_archivo)
            imagen = preprocesar_imagen(ruta_imagen)
            if imagen is None:  # Omitir imágenes no válidas
                continue
            caracteristicas = cv2.cornerHarris(imagen,2,3,0.04).flatten()
            X.append(caracteristicas)
            y.append(etiqueta)
    return np.array(X), np.array(y)

In [119]:
# 4. Clasificación
def clasificacion(y_real, y_predicha):
    nombres_clases = {0: "gatos", 1: "perros"}

    # Inicializar un diccionario para almacenar las métricas por clase
    reporte = {}
    for clase in [0, 1]:
        # Calcular las métricas por clase
        precision = precision_score(y_real, y_predicha, labels=[clase], average="micro")
        reporte[nombres_clases[clase]] = {
            'precision': precision
        }
    # Imprimir clasificación
    print("Clasificación :")
    for nombre_clase, metricas in reporte.items():
        print(f"\nClase {nombre_clase}:")
        for metrica, valor in metricas.items():
            print(f"  {metrica}: {valor * 100:.2f} %")

In [120]:
##pipeline 
train_dir = 'cat_dog_500/train'
test_dir = 'cat_dog_500/test'

print("Extrayendo características HOG...")
X_train, y_train = crear_dataset(train_dir, extraccion_hog)
X_test, y_test = crear_dataset(test_dir, extraccion_hog)

print("Entrenando modelo SVM...")
clf = SVC(kernel='linear')
clf.fit(X_train, y_train)

print("Evaluando modelo...")
y_pred = clf.predict(X_test)
clasificacion(y_test, y_pred)

Extrayendo características HOG...
Entrenando modelo SVM...
Evaluando modelo...
Clasificación :

Clase gatos:
  precision: 50.00 %

Clase perros:
  precision: 0.00 %


C:\Users\jorge\anaconda3\envs\py311ml\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
